In [2]:
from lusidtools.jupyter_tools import toggle_code

"""Bonds - Computing P&L and Accrued Interest for Bonds

Attributes
----------
bond
bonds
recipes
valuations
accrued interest
P&L
"""

toggle_code("Toggle Docstring")

# Computing P&L and Accrued Interest for Inflation Linked Bonds

In this notebook, we demonstrate how P&L and accrued interests can be computed for Inflation Linked Bond instruments as well as how to book in bond coupons to our cash position. 

## Table of Contents:
- 1. [Create a portfolio](#1.-Create-Portfolio)
- 2. [Create an Inflation Linked Bond](#2.-Create-Inflation-Linked-Bond)
- 3. [Transactions](#3.-Transactions)
- 4. [Quotes](#4.-Quotes)
- 5. [Valuation & PnL](#5.-Valuations-&-PnL)

In [3]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML
from utilities.formatting_tools import cashladder_to_df

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
# display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.10416.0


In [4]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)

In [8]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"
portfolio_code = "inflationBondPortfolio"

# 1. Create Portfolio

We must first create a portfolio to keep our inflation linked bond in, this will allow us to perform a valuation and inspect the cash flows at a later stage.

In [83]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="GBP",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'inflationBondPortfolio' because it already exists in scope 'ibor'.


# 2. Create Inflation Linked Bond

We create a function that takes in our variables and upserts an inflation linked bond to LUSID.

In [84]:
def create_il_bond(
    currency,
    payment_frequency,
    roll_convention,
    day_count_convention,
    payment_calendars,
    reset_calendars,
    settle_days,
    reset_days,
    start_date,
    maturity_date,
    dom_ccy,
    principal,
    coupon_rate,
    inflation_index,
    observation_lag,
    bond_identifier,
    bond_name,
):

    flow_conventions = lm.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=payment_calendars,
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    inflation_linked_bond = lm.InflationLinkedBond(
        start_date=start_date, #
        maturity_date=maturity_date, #
        dom_ccy=dom_ccy, #
        principal=principal, #
        coupon_rate=coupon_rate, #
        flow_conventions=flow_conventions, #
        identifiers={}, #
        instrument_type="InflationLinkedBond",
        calculation_type="Standard",
        inflation_index_name = inflation_index,
        observation_lag = observation_lag,
        index_precision = 9
    )

    # define the instrument to be upserted
    il_bond_definition = lm.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(bond_identifier)},
        definition=inflation_linked_bond,
    )

    # upsert the instrument
    upsert_request = {bond_identifier: il_bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[bond_identifier].lusid_instrument_id
    print(bond_luid)

# 2.1 Upsert a bond

We can now set the variables of the bond and upsert it into LUSID using the function we built in the cell above.

In [85]:
currency = "GBP"
payment_frequency = "6M"
roll_convention = "none"
day_count_convention = "ActualActual"
payment_calendars = []
reset_calendars = []
settle_days = 0
reset_days = 0
start_date = datetime(2009, 7, 24, 00, tzinfo=pytz.utc)
maturity_date = datetime(2042, 11, 22, 00, tzinfo=pytz.utc)
dom_ccy = "GBP"
principal = 1
coupon_rate = 0.00625
inflation_index = "UKRPI"
observation_lag = "3M"
bond_identifier = "GB00B3MYD345 UKTI 0 5/8 11/22/42"
bond_name = "UKTI 0 5/8 11/22/42"


create_il_bond(
    currency,
    payment_frequency,
    roll_convention,
    day_count_convention,
    payment_calendars,
    reset_calendars,
    settle_days,
    reset_days,
    start_date,
    maturity_date,
    dom_ccy,
    principal,
    coupon_rate,
    inflation_index,
    observation_lag,
    bond_identifier,
    bond_name,
)

LUID_00003DDB


# 3. Transactions

## 3.1 Create Transaction Request

Once the inflation linked bond has been created, we can generate a transaction that adds it to our portfolio.

In [86]:
transactions = pd.read_csv("data/il_bond_transaction_data.csv")
transactions

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency,portfolio
0,txn001,StockIn,GB00B3MYD345 UKTI 0 5/8 11/22/42,2019-04-19T10:00:00Z,2019-04-20T10:00:00Z,1000000,152.30,1523000,GBP,inflationBondPortfolio


In [87]:
for portfolio_code, grouped_df in transactions.groupby("portfolio"):

    transaction_request = [
        lm.TransactionRequest(
            transaction_id=row["txn_id"],
            type=row["type"],
            instrument_identifiers={
                "Instrument/default/ClientInternal": row["client_id"]
            },
            transaction_date=row["trade_date"],
            settlement_date=row["settlement_date"],
            units=row["quantity"],
            transaction_price=lm.TransactionPrice(price=row["price"], type="Price"),
            total_consideration=lm.CurrencyAndAmount(
                amount=row["total_consideration"], currency=row["currency"]
            ),
        )
        for index, row in grouped_df.iterrows()
    ]

    response = transaction_portfolios_api.upsert_transactions(
        scope=scope, code=portfolio_code, transaction_request=transaction_request
    )

    print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2022-12-05 17:17:24.836502+00:00


# 4. Quotes

The bond is in our portfolio, but currently lacks pricing. We will thus upsert quotes for the bond that we retrieved from our market data system. We upload prices from when the bond was purchased until today (at the time of writing, 05 December 2022). Below we can see the last 5 of the ~ 1000 quotes that were uploaded.

In [224]:
bond_prices = pd.read_csv("data/il_bond_quotes_data.csv")
bond_prices.tail()

,client_internal,date,price,currency
941,GB00B3MYD345 UKTI 0 5/8 11/22/42,29/11/2022,112.62,GBP
942,GB00B3MYD345 UKTI 0 5/8 11/22/42,30/11/2022,112.04,GBP
943,GB00B3MYD345 UKTI 0 5/8 11/22/42,01/12/2022,113.05,GBP
944,GB00B3MYD345 UKTI 0 5/8 11/22/42,02/12/2022,110.99,GBP
945,GB00B3MYD345 UKTI 0 5/8 11/22/42,05/12/2022,111.22,GBP


In [225]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["client_internal"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=to_date(row["date"]),
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
        scale_factor=100,
    )
    for index, row in bond_prices.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded.")

else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quote successfully loaded into LUSID. 946 quotes loaded.


For an inflation linked bond, we will also need the monthly index reset values. Here we upsert the UKRPI monthly resets from the inception of our bond in 2009 until today.

In [227]:
# Read in from file
resets_df = pd.read_csv("data/il_resets.csv")
resets_df.head()

,Date,Rate,Index
0,2022-08-01,345.20,UKRPI
1,2022-07-01,343.20,UKRPI
2,2022-06-01,340.00,UKRPI
3,2022-05-01,337.10,UKRPI
4,2022-04-01,334.60,UKRPI


In [139]:
# Create quotes request
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["Index"],
                    instrument_id_type="ClientInternal",
                    quote_type="Index",
                    field="mid",
            ),
            effective_at=to_date(row["Date"]),
        ),
        metric_value=lm.MetricValue(value=row["Rate"], unit="rate"),
    )
    for index, row in resets_df.iterrows()
}

# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded.")

else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quote successfully loaded into LUSID. 8 quotes loaded.


# 5. Valuations & PnL

Once we have the bond booked into a portfolio as well as the quotes and index resets upserted, we can now value it. The recipe below describes how we will go about valuing the credit instruments in our portfolio (in this case just our bond). It details which model we will be using, SimpleStatic, which is supplied by Lusid.

It also specifies where to find the Inflation Index resets under the key "Inflation.\*" and the bond's quotes under the key "Quote.\*"

## 5.1 Create valuation recipe

In [181]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="ilBondValuation",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Inflation.*.*",
                data_scope=scope,
                supplier="Lusid",
                quote_type='Index',
                field='mid',
                quote_interval="1M"
                ),
            lm.MarketDataKeyRule(
                key="Quote.*.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="10D",
            ),
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="InflationLinkedBond",
                parameters="{}",
            )
        ],
        options=lm.PricingOptions(allow_partially_successful_evaluation=True)
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create daily valuation function

Once we have made a recipe, we can now create a function that outputs a dataframe with the valuation of our portfolio. We choose to display PV, Accrued Interest and PnL as well as some instrument identifiers.

In [6]:
def get_val(date, portfolio_code):

    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code="ilBondValuation"),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Quotes/Price", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Holding/default/Accrual", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Holding/default/Accrual": "Accrued Interest",
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

We will now display the first few days in the lifecycle of our bond. Recall that we have bought this bond on Friday 19 April 2019 and this bond will only settle by Monday 22 April. Day 1 in our scenario is thus 22 April 2019.

## Day 1

At 10AM on day 1, once our bond has settled, we value our bond position with the SimpleStatic model and find that it is worth USD 1,719,499.86. This value includes about five months worth of accrued interest as the last coupon was on 2018-11-22 and we are now valuing it on 2019-04-22.

In [228]:
get_val("2019-04-22T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,719,499.86",UKTI 0 5/8 11/22/42,GB00B3MYD345 UKTI 0 5/8 11/22/42,171.60,"1,000,000.00","3,489.86",None


## Day 2

On day 2, we run our valuation again and find that our accrued interest increased and the price has moved down. Our position is now worth USD 4,339.33 less than on day 1. Our PnL after 1 day is thus USD -4,339.33 on this position.

In [229]:
get_val("2019-04-23T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,715,160.53",UKTI 0 5/8 11/22/42,GB00B3MYD345 UKTI 0 5/8 11/22/42,171.16,"1,000,000.00","3,513.79","-4,339.33"


## Coupon date

One day before the coupon date, we see the accrued interest approaching the coupon value. When the coupon date hits, the accrued interest will reset to zero.

In [10]:
get_val("2020-05-21T10:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,913,726.34",UKTI 0 5/8 11/22/42,GB00B3MYD345 UKTI 0 5/8 11/22/42,190.94,"1,000,000.00","4,276.34","15,743.91"


In [14]:
get_val("2020-05-22T01:00:00Z", portfolio_code)

,Present Value,InstrumentName,ClientInternal,Quotes/Price,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"1,897,200.00",UKTI 0 5/8 11/22/42,GB00B3MYD345 UKTI 0 5/8 11/22/42,189.72,"1,000,000.00",0.00,"-16,526.34"
